In [2]:
import pandas as pd
import numpy as np

import random
from random import randint
import string
import requests
from tqdm import tqdm

from time import sleep
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException


import warnings
warnings.filterwarnings('ignore')

In [3]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') 
opciones.add_argument('user.data-dir=selenium') 
#opciones.add_argument('--incognito')

## Extractor Groupon

In [119]:
driver = webdriver.Chrome("chromedriver.exe")
num_res= list(range(1, 1000))

dicc_groupon= { "categoria" : [],
                "nombre"    : [],
                "direccion" : [],
                "url"       : [],
                "precio"    : [],
                "precio_2"  : [],
                "rating"    : []                              
               }

categorias= ["juegos-de-escape", 
             "cursos-fotorgrafia", 
             "clases-baile", 
             "v-cata-de-vinos",
             "v-espectaculos",
             "carreras",
             "bolos",
             "paintball",
             "equitacion",
             "v-escalada",
             "v-esqui-y-snowboard",
             "deportes-acuaticos",
             "patinaje-sobre-hielo",
             "v-golf",
             "parapente",
             "tours-de-vinos",
             "excursiones-por-la-ciudad-y-lugares-de-interes",
             "excursiones-y-aventuras-al-aire-libre",
             "tour-de-vuelo",
             "day-spa"
             ]


for categoria in categorias:

    driver.get(f"https://www.groupon.es/ofertas/{categoria}?distance=%5B0.0..50.0%5D")

    try:
        driver.find_element("xpath", '//*[@id="nothx"]').click()
        sleep(6)
    except:
        pass

    try:
        driver.find_element("xpath", '//*[@id="gdpr-accept"]').click()
        sleep(3)
    except:
        pass

    for i in tqdm(num_res):
        try:

            dicc_groupon["nombre"].append(driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a/div[2]/div[1]/div').text)
            dicc_groupon["direccion"].append(driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a/div[2]/div[2]/div/span[1]').text)
            dicc_groupon["url"].append(driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a').get_attribute('href'))
            dicc_groupon["categoria"].append(categoria)

            try:
                dicc_groupon["precio"].append((driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a/div[2]/div[4]/div[1]/div[1]/div[2]/span').text))

            except:
                dicc_groupon["precio"].append(np.nan)

            try:
                dicc_groupon["precio_2"].append(driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a/div[2]/div[4]/div[1]/div[2]/div[2]/div[2]').text)

            except:
                dicc_groupon["precio_2"].append(np.nan)

            try:
                dicc_groupon["rating"].append(driver.find_element("xpath", f'//*[@id="pull-deal-feed"]/figure[{i}]/div/a/div[2]/div[3]/div/div[1]').text)
            except:
                dicc_groupon["rating"].append(np.nan)


            sleep(randint(3,5))

        except NoSuchElementException:
            print(f"Fin {categoria}")
            print(f"Element not found {i}")
            break


df_groupon= pd.DataFrame(dicc_groupon)
df_groupon.to_csv(f'../../data/data_inicial/GROUPON.csv', header=True, index=False)

  1%|          | 7/999 [00:30<1:11:12,  4.31s/it]

Fin cursos-fotorgrafia
Element not found 8


## Extractor Groupon Detalle

In [4]:
df= pd.read_csv('../../data/data_inicial/GROUPON.csv')

In [44]:
lista_url=list(df["url"].unique())

In [46]:
driver = webdriver.Chrome("chromedriver.exe")

dicc_groupon_detalle= { "url"        :[],
                        "descripcion":[],
                        "imagen"     :[]
                      }


for url in tqdm(lista_url):
    
    driver.get(url)

    try:
        driver.find_element("xpath", '//*[@id="nothx"]').click()
        sleep(6)
    except:
        pass

    try:
        driver.find_element("xpath", '//*[@id="gdpr-accept"]').click()
        sleep(3)
    except:
        pass

    try:

        dicc_groupon_detalle["url"].append(url)

        try:
            dicc_groupon_detalle["imagen"].append(driver.find_element("xpath", f'//*[@id="overflow-container"]/div/div/img').get_attribute("src"))
        except:
            dicc_groupon_detalle["imagen"].append(np.nan)

        try:
            dicc_groupon_detalle["descripcion"].append(driver.find_element("xpath", f'//*[@id="deal-pitch"]').text)
        except:
            dicc_groupon_detalle["descripcion"].append(np.nan)

        sleep(randint(3,5))

    except NoSuchElementException:
        print(f"Element not found {url}")
        break

100%|██████████| 88/88 [08:33<00:00,  5.84s/it]


In [52]:
df_groupon_detalle= pd.DataFrame(dicc_groupon_detalle)
df_groupon_detalle.to_csv(f'../../data/data_inicial/GROUPON_DETALLE.csv', header=True, index=False)